In [0]:
from pyspark import SparkContext

In [0]:
rdd1=sc.textFile('/FileStore/tables/5000_Sales_Records.csv',2)

In [0]:
header =rdd1.first() 

In [0]:
rdd2 = rdd1.filter(lambda row:row != header) 

In [0]:
def frmt_dt(dt):
    updt=dt.split('/')[2]+dt.split('/')[0]+dt.split('/')[1]
    return int(updt)

In [0]:
rdd3=rdd2.map(lambda x:(x.split(',')[0],x.split(',')[1],x.split(',')[2],x.split(',')[3],x.split(',')[4],frmt_dt(x.split(',')[5]),x.split(',')[6],frmt_dt(x.split(',')[7]),int(x.split(',')[8]),float(x.split(',')[9]),float(x.split(',')[10]),float(x.split(',')[11]),float(x.split(',')[12]),float(x.split(',')[13])))

In [0]:
# 1. Display country wise number of orders
country_orders=rdd3.map(lambda x:(x[1],x[6])).groupBy(lambda x:x[0]).map(lambda x:(x[0],len(x[1])))

In [0]:
print(country_orders.take(15))

[('Antigua and Barbuda ', 26), ('North Korea', 24), ('Federated States of Micronesia', 20), ('Ethiopia', 26), ('Saint Lucia', 29), ('Lebanon', 31), ('Austria', 28), ('Mexico', 23), ('Trinidad and Tobago', 19), ('Libya', 32), ('Algeria', 28), ('Estonia', 23), ('Saudi Arabia', 23), ('Montenegro', 28), ('Guatemala', 28)]


In [0]:
# 2.Display the number of units sold in each region.
regions_units=rdd3.map(lambda x:(x[0],x[8])).reduceByKey(lambda x,y:x+y)

In [0]:
print(regions_units.take(15))

[('Asia', 3620036), ('Middle East and North Africa', 3013431), ('Australia and Oceania', 2111786), ('Central America and the Caribbean', 2698776), ('Europe', 6582322), ('Sub-Saharan Africa', 6642380), ('North America', 484760)]


In [0]:
# 3.Display the 10 most recent sales. 
from datetime import datetime


In [0]:
rd5= rdd3.map(lambda x : (x[6],datetime(int(x[5]))))

In [0]:
rd5 = rdd3.map(lambda row: row[:5] + [datetime.strptime(row[5],'%m/%d/%Y')] + row[6:])

In [0]:
rd5.take(2)

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-1594355762469396> in <cell line: 1>()
----> 1 rd5.take(2)

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 res = func(*args, **kwargs)
     49                 logger.log_success(
     50                     module_name, class_name, function_name, time.perf_counter() - start, signature

/databricks/spark/python/pyspark/rdd.py in take(self, num)
   1923 
   1924             p = range(partsScanned, min(partsScanned + numPartsToTry, totalParts))
-> 1925             res = self.context.runJob(self, takeUpToNumLeft, p)
   1926 
   1927             items += res

/databricks/spark/python/pyspark/context.py in runJob(self, rdd, partitionFunc, partitions, allowLocal)
   1643             finally:
   1644   

In [0]:
# 4.Display the products with atleast 2 occurences of 'a' (Using spark)
products_a_count=rdd3.map(lambda x:x[2]).filter(lambda x:x.count('a')>=2)

In [0]:
print(products_a_count.take(5))

['Personal Care', 'Personal Care', 'Personal Care', 'Personal Care', 'Personal Care']


In [0]:
# 5.Display country in each region with highest units sold. (Using spark)

In [0]:
country_sales = rdd3.map(lambda row : ((row[0] , row[1]),int(row[8])))
c_reduced = country_sales.reduceByKey(lambda a,b :a+b)


In [0]:
print(c_reduced.map(lambda x: (x[0][0],(x[0][1],x[1]))).reduceByKey(lambda a,b : max(a,b ,key=lambda x : x[1])).take(10))

[('Asia', ('Myanmar', 199967)), ('Australia and Oceania', ('Australia', 183909)), ('Middle East and North Africa', ('Somalia', 193065)), ('Central America and the Caribbean', ('Grenada', 205943)), ('Europe', ('Macedonia', 203078)), ('Sub-Saharan Africa', ('Equatorial Guinea', 197767)), ('North America', ('United States of America', 159519))]


In [0]:
# 6.Display the unit price and unit cost of each item in ascending order. (Using spark)
item_cost=rdd3.map(lambda x:(x[2],x[9],x[10])).distinct().sortBy(lambda x:x[2])

In [0]:
print(item_cost.take(20))

[('Fruits', 9.33, 6.92), ('Beverages', 47.45, 31.79), ('Clothes', 109.28, 35.84), ('Personal Care', 81.73, 56.67), ('Vegetables', 154.06, 90.93), ('Snacks', 152.58, 97.44), ('Cereal', 205.7, 117.11), ('Baby Food', 255.28, 159.42), ('Cosmetics', 437.2, 263.33), ('Meat', 421.89, 364.69), ('Household', 668.27, 502.54), ('Office Supplies', 651.21, 524.96)]


In [0]:
# 7.Display the number of sales yearwise. (Using pyspark)
yearwise_sales=rdd3.map(lambda x:(str(x[5])[:4],x[6])).groupBy(lambda x:x[0]).map(lambda x:(x[0],len(x[1])))

In [0]:
print(yearwise_sales.take(10))

[('2013', 660), ('2010', 632), ('2011', 658), ('2015', 679), ('2016', 670), ('2014', 660), ('2017', 363), ('2012', 678)]


In [0]:
# 8.Display the number of orders for each item. (Using pyspark)
item_orders=rdd3.map(lambda x:(x[2],x[6])).groupBy(lambda x:x[0]).map(lambda x:(x[0],len(x[1])))

In [0]:
print(item_orders.take(15))

[('Baby Food', 445), ('Snacks', 398), ('Cereal', 385), ('Clothes', 386), ('Cosmetics', 424), ('Fruits', 447), ('Beverages', 447), ('Personal Care', 415), ('Office Supplies', 420), ('Meat', 399), ('Vegetables', 410), ('Household', 424)]


In [0]:
# 9.Display the countr.with highest calallicing pyspark)